In [79]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [87]:
##Question 1

def generate_correlated_variables(n, corr):
    """
    Generates two variables X and Y such that they have the specified correlation.
    
    Parameters:
        n: Number of samples for each variable.
        corr: Desired correlation between X and Z.
    
    Returns:
        tuple: Two numpy arrays X and Z.
    """
    # Generate two i.i.d. N(0,1) variables, X and Y
    X = np.random.normal(0, 1, n)
    Y = np.random.normal(0, 1, n)

    # Create Z to have the specified correlation with X
    Z = corr * X + np.sqrt(1 - corr**2) * Y

    return X, Z

# Number of samples
n_samples = 5000

# Desired correlation
rho = 0.5

# Generate the variables
X, Z = generate_correlated_variables(n_samples, rho)

Q1_data = pd.DataFrame({'Variable 1':X,'Variable 2':Z})
Q1_data

,Variable 1,Variable 2
0,0.320693,-0.242482
1,-0.397981,-0.673748
2,-0.641372,0.339958
3,-0.573893,1.243721
4,-1.127619,0.615855
...,...,...
4995,0.065645,-0.670974
4996,-1.522337,-1.513431
4997,1.430072,0.126043
4998,-0.102350,-1.285643


In [94]:
##Question 2

sp500 = pd.read_excel("Sample Data in Question.xlsx", sheet_name="Q2 SP500")
fx = pd.read_excel("Sample Data in Question.xlsx", sheet_name="Q2 FX")
# Merge the two dataframes
Q2_data = pd.merge(sp500, fx, on='Date')
Q2_data = Q2_data.sort_values('Date').reset_index(drop=True)

mean_fx = np.mean(Q2_data['FX'])
mean_sp500 = np.mean(Q2_data['SP500'])
std_fx = np.std(Q2_data['FX'])
std_sp500 = np.std(Q2_data['SP500'])
Q2_data['FX_z'] = (Q2_data['FX'] - mean_fx) / std_fx
Q2_data['SP500_z'] = (Q2_data['SP500'] - mean_sp500) / std_sp500

rho=0.5
Q2_data['Z_norm'] = rho * Q2_data['FX_z'] + np.sqrt(1 - rho**2) * Q2_data['SP500_z']

# Calculate the correlation
mean_Z = rho *mean_fx + np.sqrt(1 - rho**2) * mean_sp500
std_Z = np.sqrt((rho **2 * std_fx**2) + ((1 - rho**2) * std_sp500**2))

Q2_data['Z'] = mean_Z + std_Z * Q2_data['Z_norm'] # Z and FX are correlated with rho

with pd.ExcelWriter('Sample Data in Question.xlsx', mode='a', engine='openpyxl') as writer:
    # Write the DataFrame to a new tab in the Excel file
    Q2_data.to_excel(writer, sheet_name='Q2 output', index=False)


In [98]:
##Question 3
def generate_multivariate_normal(n, corr_matrix):
    """
    Generates multivariate normal data with the specified correlation matrix.
    Parameters:
        n: Number of samples for each variable.
        corr_matrix: Desired correlation matrix.
    Returns:
        Z: Generated data with desired correlations.
    """
    # Cholesky decomposition of the correlation matrix
    L = np.linalg.cholesky(corr_matrix)
    # Generate a matrix of i.i.d. N(0,1) variables
    X = np.random.normal(0, 1, (n, len(corr_matrix)))
    # Apply the linear transformation
    Z = X @ L.T
    return Z
    
# Define the correlation matrix
R = np.array([
[1, 0.5, 0.5, 0.5],
[0.5, 1, 0.5, 0.5],
[0.5, 0.5, 1, 0.5],
[0.5, 0.5, 0.5, 1]
])

# Number of samples
n_samples = 5000

# Generate the data
Q3_data = generate_multivariate_normal(n_samples, R)
Q3_data =pd.DataFrame(Q3_data)

with pd.ExcelWriter('Sample Data in Question.xlsx', mode='a', engine='openpyxl') as writer:
    # Write the DataFrame to a new tab in the Excel file
    Q3_data.to_excel(writer, sheet_name='Q3 output', index=False)

In [46]:
##Question 5

Q5_data1 = pd.read_excel("Sample Data in Question.xlsx", sheet_name="Q5 data1")
Q5_data2 = pd.read_excel("Sample Data in Question.xlsx", sheet_name="Q5 data2")

#align up the two timeseries by 'Date'
Q5_data = pd.merge(Q5_data1,Q5_data2,on='Date', how='outer')

#fill the missing value

#method 1: through linear interpolation
Q5_data['Close_x']=Q5_data['Close_x'].interpolate(method='linear')
print('filling missing value through linear interpolation\n',Q5_data)

#method 2: by carrying forward the last value
Q5_data = pd.merge(Q5_data1,Q5_data2,on='Date', how='outer')
Q5_data['Close_x']=Q5_data['Close_x'].ffill()
print('\nfilling missing value through forward filling\n',Q5_data)

filling missing value through linear interpolation
         Date   Close_x  Close_y
0 2019-12-31  3230.780  1.30606
1 2020-01-01  3244.315  1.30020
2 2020-01-02  3257.850  1.29730
3 2020-01-03  3234.850  1.29830
4 2020-01-06  3246.280  1.29866

filling missing value through forward filling
         Date  Close_x  Close_y
0 2019-12-31  3230.78  1.30606
1 2020-01-01  3230.78  1.30020
2 2020-01-02  3257.85  1.29730
3 2020-01-03  3234.85  1.29830
4 2020-01-06  3246.28  1.29866


In [77]:
##Question 6

Q6_data = pd.read_excel("Sample Data in Question.xlsx", sheet_name="Q6")
Q6_data=Q6_data.sort_values(by='Date')

# Drop rows with missing DV01 Convexity values (for training)
Q6_train = Q6_data.dropna(subset=['DV01 Convexity'])

# Extract features (DV01 and MV) and target (DV01 Convexity) from training data
X_train = Q6_train[['DV01', 'MV']]
y_train = Q6_train['DV01 Convexity']

# Fit linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Use trained model to predict missing DV01 Convexity values
X_pred = Q6_data[Q6_data['DV01 Convexity'].isnull()][['DV01', 'MV']]
y_pred = model.predict(X_pred)

#Validate the accuracy of the regression model by comparing to the known DV01 Convexity
y_validate=model.predict(X_train)
Q6_data.loc[Q6_data['DV01 Convexity'].notnull(), 'DV01 Convexity Validate'] = y_validate

# Fill missing DV01 Convexity values with predictions
Q6_data.loc[Q6_data['DV01 Convexity'].isnull(), 'DV01 Convexity'] = y_pred

print('Q6 with DV01 Convexity predicted through linear regression\n',Q6_data)



Q6 with DV01 Convexity predicted through linear regression
         Date Portfolio         MV    DV01  DV01 Convexity  \
0 2023-10-27      FICA   67454088  158800      493.047331   
1 2023-10-30      FICA   89304605  230212      661.516314   
2 2023-10-31      FICA  113449488  252102      700.838898   
6 2023-11-06      FICA  183099003  399745     1032.000000   
3 2023-11-07      FICA  185055847  404225     1042.000000   
5 2023-11-10      FICA  182731405  398644     1029.000000   
4 2023-11-13      FICA  182724462  398461     1029.000000   

   DV01 Convexity Validate  
0                      NaN  
1                      NaN  
2                      NaN  
6              1031.865676  
3              1042.020716  
5              1029.289979  
4              1028.823629  


In [117]:
#Question 8

FX = pd.read_excel("Q8 Data.xlsx", sheet_name="USDCAD")
SP500 = pd.read_excel("Q8 Data.xlsx", sheet_name="SPX 500")
TSX = pd.read_excel("Q8 Data.xlsx", sheet_name="S&P TSX")
GSCI = pd.read_excel("Q8 Data.xlsx", sheet_name="S&P GSCI Index ")
Gold = pd.read_excel("Q8 Data.xlsx", sheet_name="Gold")

Q8_data = pd.merge(FX, SP500, on='Date', how='outer')
Q8_data = pd.merge(Q8_data, TSX, on='Date', how='outer')
Q8_data = pd.merge(Q8_data, GSCI, on='Date', how='outer')
Q8_data = pd.merge(Q8_data, Gold, on='Date', how='outer')

Q8_data=Q8_data.dropna()

with pd.ExcelWriter('Q8 Data.xlsx', mode='a', engine='openpyxl') as writer:
    # Write the DataFrame to a new tab in the Excel file
    Q8_data.to_excel(writer, sheet_name='Q8 Data', index=False)


In [135]:
##Question 12
sp500 = pd.read_excel("Sample Data in Question.xlsx", sheet_name="SPX 500")
sp500 = sp500.sort_values('Date').reset_index(drop=True)

# Calculate daily returns
sp500['Daily Returns'] = sp500['S&P500'].pct_change()
sp500['Daily Returns'] = np.log(sp500['Daily Returns'])

# Calculate the standard deviation of daily returns
daily_volatility = sp500['Daily Returns'].std()

# Annualize the volatility
annualized_volatility = daily_volatility * np.sqrt(252)
print(annualized_volatility)

18.025130738350875


D:\Python\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
